In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [2]:
print(tf.__version__)

2.17.0


In [3]:
# Load the two models
model1 = tf.keras.models.load_model('xception_train1.h5')
model2 = tf.keras.models.load_model('xception_train2.h5')
model3 = tf.keras.models.load_model('xception_train3.h5')
model4 = tf.keras.models.load_model('xception_train4.h5')

# Ensure that both models have the same architecture
assert model1.count_params() == model2.count_params(), "Models do not have the same architecture!"

# Extract weights from both models
weights1 = model1.get_weights()
weights2 = model2.get_weights()
weights3 = model2.get_weights()
weights4 = model2.get_weights()

In [5]:
print(f"Model 1 weights: {len(weights1)}")
print(f"Model 2 weights: {len(weights2)}")
print(f"Model 3 weights: {len(weights3)}")
print(f"Model 4 weights: {len(weights4)}")


Model 1 weights: 238
Model 2 weights: 238
Model 3 weights: 238
Model 4 weights: 238


In [125]:
# Ensure that both models have the same architecture
assert model1.count_params() == model2.count_params(), "Models do not have the same architecture!"

# Extract weights from both models
weights1 = model1.get_weights()
weights2 = model2.get_weights()
weights3 = model3.get_weights()
weights4 = model4.get_weights()

# Initialize a new model by copying the architecture and weights from model1
merged_model = tf.keras.models.clone_model(model1)
merged_model.set_weights(weights1)

# Accuracies of the models
acc1, acc2, acc3, acc4 = 0.66, 0.87, 0.82, 0.83
# Normalize the accuracies to get weights
total_acc = acc1 + acc2 + acc3 + acc4
wx1 = acc1 / total_acc
wx2 = acc2 / total_acc
wx3 = acc3 / total_acc
wx4 = acc4 / total_acc

# n = 0.05 +0.6+0.15+0.15 # Experimental value
# n = 0.795
n = 0.97


average_weights = [((w1*wx1) + (w2*wx2) + (w3*wx3) + (w4*wx4)) / n for w1, w2, w3, w4 in zip(weights1, weights2, weights3, weights4)]

merged_model.set_weights(average_weights)

# Save the merged model
merged_model.save('merged_model.h5')

print("Merged model saved successfully.\nQuantized model saved successfully!!!")

Merged model saved successfully.


In [126]:
# Load and preprocess the dataset for testing
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

test_generator = test_datagen.flow_from_directory(
    r"split_chest_xray\test",  # Replace with the path to your testing dataset directory
    target_size=(224, 224),
    # batch_size=32,
    batch_size=64,
    class_mode='categorical'
)

Found 624 images belonging to 2 classes.


In [127]:
model3 = tf.keras.models.load_model('merged_model.h5')
model3.compile(optimizer='adam',  # or your preferred optimizer
               loss='categorical_crossentropy',  # or your preferred loss function
               metrics=['accuracy'])  # or your preferred metrics


In [128]:
# Evaluate the model on the testing data
test_loss, test_accuracy = model3.evaluate(test_generator)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 37s 3s/step - accuracy: 0.8784 - loss: 0.3817
Test Loss: 0.3796602189540863, Test Accuracy: 0.8894230723381042


In [6]:
import os

def get_size(file_path):
    # Get the file size in bytes
    file_size_bytes = os.path.getsize(file_path)
    
    # Convert bytes to megabytes (1 MB = 1024 * 1024 bytes)
    file_size_mb = file_size_bytes / (1024 * 1024)
    
    return f"The size of the file '{file_path}' is: {file_size_mb:.2f} MB\n"



In [7]:
print(get_size(r"xception_train_entire_dataset.h5"))
print(get_size(r"xception_train1.h5"))
print(get_size(r"xception_train2.h5"))
print(get_size(r"xception_train3.h5"))
print(get_size(r"xception_train3.h5"))

print("\n\nNew MERGED FILE:",get_size(r"merged_model.h5"))

The size of the file 'xception_train_entire_dataset.h5' is: 262.94 MB

The size of the file 'xception_train1.h5' is: 262.94 MB

The size of the file 'xception_train2.h5' is: 262.94 MB

The size of the file 'xception_train3.h5' is: 262.94 MB

The size of the file 'xception_train3.h5' is: 262.94 MB



New MERGED FILE: The size of the file 'merged_model.h5' is: 88.02 MB

